# [Advent of Code 2019: Day 8](https://adventofcode.com/2019/day/8)

<h2>--- Day 8: Space Image Format ---</h2>
<p><span title="I'm not sorry.">The Elves' spirits are lifted when they realize you have an opportunity to reboot one of their Mars rovers, and so they are curious if you would spend a brief sojourn on Mars.</span> You land your ship near the rover.</p>
<p>When you reach the rover, you discover that it's already in the process of rebooting! It's just waiting for someone to enter a <a href="https://en.wikipedia.org/wiki/BIOS">BIOS</a> password. The Elf responsible for the rover takes a picture of the password
    (your puzzle input) and sends it to you via the Digital Sending Network.</p>
<p>Unfortunately, images sent via the Digital Sending Network aren't encoded with any normal encoding; instead, they're encoded in a special Space Image Format. None of the Elves seem to remember why this is the case. They send you the instructions to decode
    it.</p>
<p>Images are sent as a series of digits that each represent the color of a single pixel. The digits fill each row of the image left-to-right, then move downward to the next row, filling rows top-to-bottom until every pixel of the image is filled.</p>
<p>Each image actually consists of a series of identically-sized <em>layers</em> that are filled in this way. So, the first digit corresponds to the top-left pixel of the first layer, the second digit corresponds to the pixel to the right of that on the
    same layer, and so on until the last digit, which corresponds to the bottom-right pixel of the last layer.</p>
<p>For example, given an image <code>3</code> pixels wide and <code>2</code> pixels tall, the image data <code>123456789012</code> corresponds to the following image layers:</p>
<pre><code>Layer 1: 123
         456

Layer 2: 789
         012
</code></pre>
<p>The image you received is <em><code>25</code> pixels wide and <code>6</code> pixels tall</em>.</p>
<p>To make sure the image wasn't corrupted during transmission, the Elves would like you to find the layer that contains the <em>fewest <code>0</code> digits</em>. On that layer, what is <em>the number of <code>1</code> digits multiplied by the number of <code>2</code> digits?</em></p>

In [1]:
import os
import unittest

class SpaceImage():
    
    BLACK = '0'
    WHITE = '1'
    TRANS = '2'
    
    def __init__(self, width, height, stream):
        self.layer_width = width
        self.layer_height = height
        self.data = []
        self.load_data(stream)
        
    def load_data(self, data_stream):
        if type(data_stream) is int:
            data_stream = str(data_stream)
            
        step = self.layer_width * self.layer_height
        self.data = self.step_slice(step, data_stream)
            
    def step_slice(self, step, data):
        sliced_data = []
        for i in range(0, len(data), step):
            sliced_data.append(data[i:i+step])
        return sliced_data
        
    def validate_data(self):
        
        layer_zeros = []
        
        for layer in self.data:
            layer_zeros.append(layer.count("0"))
            
        target_layer = self.data[layer_zeros.index(min(layer_zeros))]
        
        ones = target_layer.count("1")
        twos = target_layer.count("2")
        
        return ones * twos
    
    def merge_layers(self):
        img = None
        for layer in self.data:
            if img != None:
                for i in range(0, len(layer)):
                    if img[i] is self.TRANS:    
                        img[i] = layer[i]
            else:
                img = list(layer)
        img = "".join(img)
        return img
    
    def decode(self):
        msg_data = self.merge_layers()
        msg_data = msg_data.replace('0', ' ')
        msg_data = msg_data.replace('1', '#')
        msg_rows = self.step_slice(self.layer_width, msg_data)
        msg = '\n'.join(msg_rows)
        return msg
        
    
class TestBasic(unittest.TestCase):

    def test_space_image(self):
        data = [("0222112222120000",'0110')]
        for input, output in data:
            si = SpaceImage(width = 2, height = 2, stream = input)
            self.assertEqual(output ,si.merge_layers())

unittest.main(argv=[""], exit=False)      

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


In [2]:
with open("inputs/input_d08.txt") as file:
    input_stream = file.read()
    
si = SpaceImage(width = 25, height = 6, stream = input_stream)
si.validate_data()

1441

<h2 id="part2">--- Part Two ---</h2>
<p>Now you're ready to decode the image. The image is rendered by stacking the layers and aligning the pixels with the same positions in each layer. The digits indicate the color of the corresponding pixel: <code>0</code> is black, <code>1</code> is white,
    and <code>2</code> is transparent.</p>
<p>The layers are rendered with the first layer in front and the last layer in back. So, if a given position has a transparent pixel in the first and second layers, a black pixel in the third layer, and a white pixel in the fourth layer, the final image
    would have a <em>black</em> pixel at that position.</p>
<p>For example, given an image <code>2</code> pixels wide and <code>2</code> pixels tall, the image data <code>0222112222120000</code> corresponds to the following image layers:</p>
<pre><code>Layer 1: <em>0</em>2
         22

Layer 2: 1<em>1</em>
         22

Layer 3: 22
         <em>1</em>2

Layer 4: 00
         0<em>0</em>
</code></pre>
<p>Then, the full image can be found by determining the top visible pixel in each position:</p>
<ul>
    <li>The top-left pixel is <em>black</em> because the top layer is <code>0</code>.</li>
    <li>The top-right pixel is <em>white</em> because the top layer is <code>2</code> (transparent), but the second layer is <code>1</code>.</li>
    <li>The bottom-left pixel is <em>white</em> because the top two layers are <code>2</code>, but the third layer is <code>1</code>.</li>
    <li>The bottom-right pixel is <em>black</em> because the only visible pixel in that position is <code>0</code> (from layer 4).</li>
</ul>
<p>So, the final image looks like this:</p>
<pre><code>01
10
</code></pre>
<p><em>What message is produced after decoding your image?</em></p>

In [3]:
print(si.decode())

###  #  # #### ###  ###  
#  # #  #    # #  # #  # 
#  # #  #   #  ###  #  # 
###  #  #  #   #  # ###  
# #  #  # #    #  # #    
#  #  ##  #### ###  #    
